<a href="https://colab.research.google.com/github/Ygrick/colab/blob/main/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ромайкин Игорь. ФПИИТ-03-21


1. https://www.kaggle.com/pcbreviglieri/recommending-music-artists-with-boltzmann-machines

2. https://makesomecode.me/2018/09/movie-recommendation-system/

3. https://habr.com/en/company/lanit/blog/421401/

4. https://habr.com/en/company/ivi/blog/232843/

5. https://www.alpha-quantum.com/blog/collaborative-filtering-recommender-systems/collaborative-filtering-recommender-system-with-python-from-scratch-using-svd-item-based-model-based-approaches/

6. https://predictivehacks.com/item-based-collaborative-filtering-in-python/

7. https://www.dmitrymakarov.ru/intro/recommender-17/


In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_of_file_artists = '/content/drive/MyDrive/lastfm_artist_list.csv'
path_of_file_scrobbles = '/content/drive/MyDrive/lastfm_user_scrobbles.csv'

In [ ]:
df_artists = pd.read_csv(path_of_file_artists)
df_artists

,artist_id,artist_name
0,1,__Max__
1,2,_Algol_
2,3,-123 Min.
3,4,-Oz-
4,5,-T De Sangre
...,...,...
17488,17489,鷺巣詩郎
17489,17490,黃立行
17490,17491,黄义达
17491,17492,黒木メイサ


In [ ]:
df_scrobbles = pd.read_csv(path_of_file_scrobbles)
df_scrobbles

,user_id,artist_id,scrobbles
0,1,4562,13883
1,1,10191,11690
2,1,494,11351
3,1,6673,10300
4,1,8402,8983
...,...,...,...
92787,1892,10098,278
92788,1892,8660,263
92789,1892,3274,258
92790,1892,4240,232


In [ ]:
df_merge_users_and_artist = df_scrobbles.merge(df_artists)
df_merge_users_and_artist

,user_id,artist_id,scrobbles,artist_name
0,1,4562,13883,Duran Duran
1,3,4562,228,Duran Duran
2,25,4562,85,Duran Duran
3,26,4562,10,Duran Duran
4,60,4562,528,Duran Duran
...,...,...,...,...
92787,1892,10879,337,Nyktalgia
92788,1892,1357,297,Atsakau Niekada
92789,1892,4360,281,Domantas Razauskas
92790,1892,1330,280,Atalyja


In [ ]:
#создание матрицы предпочтений
_, uniq_user =  np.unique(df_scrobbles['user_id'].values, return_inverse=True) 
_, uniq_artists = np.unique(df_scrobbles['artist_id'].values, return_inverse=True)
values_scrobbles = df_scrobbles['scrobbles'].values
user_item_matrix = sparse.csr_matrix((values_scrobbles, (uniq_user, uniq_artists)))
user_item_matrix

<1892x17493 sparse matrix of type '<class 'numpy.longlong'>'
	with 92723 stored elements in Compressed Sparse Row format>

In [ ]:
# from scipy.sparse import csc_matrix
# from scipy.sparse.linalg import svds, eigs
# u, s, vt = svds(A, k=2)

In [ ]:
# поиск ближайших соседей (наиболее похожих пользователей)
# метрика - косинусное расстояние между пользователями
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)
res = knn.fit(user_item_matrix)

In [ ]:
# выбор пользователя, которому будем давать рекомендации
id_of_user = 10
distances, indices = res.kneighbors(user_item_matrix[id_of_user], 
                                    n_neighbors = 10)

In [ ]:
# просчёт дистанции до других пользователей
df_distances = pd.DataFrame(data = distances.squeeze().tolist(), 
                            columns=['distance'])

df_indices = pd.DataFrame(data = indices.squeeze().tolist(), 
                          columns=['user_id'])

df_distances_to_user = pd.concat([df_distances, 
                                  df_indices], 
                                 axis=1)

In [ ]:
df_distances_to_user = df_distances_to_user.sort_values(by='distance')
df_distances_to_user

,distance,user_id
0,1.110223e-16,10
1,4.608656e-01,691
2,4.733685e-01,1455
3,5.107479e-01,176
4,5.368441e-01,179
5,5.480341e-01,924
6,5.673605e-01,819
7,5.788841e-01,943
8,5.891691e-01,1488
9,5.943923e-01,864


In [ ]:
# Выводим слушаемых исполнителей для каждого юзера
# Убираем самого себя из этого списка
df_nearest_users = df_distances_to_user.merge(df_merge_users_and_artist, how='left')
df_nearest_users = df_nearest_users.loc[df_nearest_users['user_id'] != id_of_user]
df_nearest_users = df_nearest_users.reset_index(drop=True)
df_nearest_users

,distance,user_id,artist_id,scrobbles,artist_name
0,0.460866,691,9496,279,Marisa Monte
1,0.460866,691,9438,179,Maria Bethania
2,0.460866,691,857,125,Ana Carolina
3,0.460866,691,1820,135,Beyonce
4,0.460866,691,8014,95,Katy Perry
...,...,...,...,...,...
397,0.594392,864,3911,368,Deepspace
398,0.594392,864,15879,350,Two Lone Swordsmen
399,0.594392,864,2422,323,Brothomstates
400,0.594392,864,14706,296,The Ghost Orchid


In [ ]:
# среди этой группы выбираем 10 наиболее прослушиваемых исполнителей
df_artists_and_scrobbles = df_nearest_users.iloc[:,2:4]
df_artists_and_scrobbles_groupby = ((df_artists_and_scrobbles.groupby(['artist_id'], as_index=False).sum()).sort_values(by='scrobbles', ascending=False)).head(20)
df_artists_and_scrobbles_groupby.head(8)

,artist_id,scrobbles
56,2381,11580
202,9226,10209
187,8402,10014
188,8460,9110
323,15299,8373
42,1820,6603
197,8989,6061
166,7493,5159


In [ ]:
df_artists_and_scrobbles_groupby_with_names = df_artists_and_scrobbles_groupby.merge(df_artists, how='left')
df_artists_and_scrobbles_groupby_with_names.head(10)

,artist_id,scrobbles,artist_name
0,2381,11580,Britney Spears
1,9226,10209,Madonna
2,8402,10014,Kylie Minogue
3,8460,9110,Lady Gaga
4,15299,8373,The Used
5,1820,6603,Beyonce
6,8989,6061,Lostprophets
7,7493,5159,Joan Baez
8,12276,4588,Rihanna
9,13776,4312,Story Of The Year


In [ ]:
df_artists_and_scrobbles_groupby_with_names

,artist_id,scrobbles,artist_name
0,2381,11580,Britney Spears
1,9226,10209,Madonna
2,8402,10014,Kylie Minogue
3,8460,9110,Lady Gaga
4,15299,8373,The Used
5,1820,6603,Beyonce
6,8989,6061,Lostprophets
7,7493,5159,Joan Baez
8,12276,4588,Rihanna
9,13776,4312,Story Of The Year


In [ ]:
df_merge_users_and_artist = df_merge_users_and_artist[df_merge_users_and_artist['user_id'] == id_of_user]

# берём рекомендательный список и список тех, кого юзер уже слушал
list_recomended_artists = df_artists_and_scrobbles_groupby_with_names['artist_name'].values 
list_self_artists = df_merge_users_and_artist['artist_name'].values

In [ ]:
df_merge_users_and_artist.head(8)

,user_id,artist_id,scrobbles,artist_name
1417,10,9226,229,Madonna
2754,10,8460,3384,Lady Gaga
4331,10,9867,1007,Michael Jackson
10301,10,6145,329,Green Day
10727,10,10735,370,Nirvana
12155,10,12276,749,Rihanna
12640,10,2381,3851,Britney Spears
13512,10,3068,555,Christina Aguilera


In [ ]:
# убираем из рекомендательного тех, кого уже слушали
recomended_artists = [el 
                      for el in list_recomended_artists 
                      if not el in list_self_artists]
recomended_artists

['Kylie Minogue',
 'The Used',
 'Lostprophets',
 'Joan Baez',
 'Story Of The Year',
 'Trapt',
 'Mariah Carey',
 'Drowning Pool',
 'Red',
 'Rise Against',
 'Whitney Houston',
 'Ciara',
 'The Red Jumpsuit Apparatus',
 '30 Seconds To Mars']

In [ ]:
len(recomended_artists)

14